In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pickle
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import ipywidgets as widgets
from IPython.display import display
import warnings
import os
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('../data/dataset_modelagem.csv')
df['date'] = pd.to_datetime(df['date'])

In [ ]:
model_files = [f for f in os.listdir('../models/') if f.endswith('.pkl')]
latest_model = sorted(model_files)[-1]
print(f"Carregando modelo: {latest_model}")

with open(f'../models/{latest_model}', 'rb') as f:
    model = pickle.load(f)

In [ ]:
features = ['consumption_lag1', 'consumption_lag7', 'consumption_ma7']

mask = ~df[features + ['target']].isnull().any(axis=1)
df_clean = df[mask].copy()

df_clean = df_clean.sort_values('date')
split_idx = int(len(df_clean) * 0.7)
df_test = df_clean.iloc[split_idx:].copy()

X_test = df_test[features]
y_true = df_test['target'].values
dates_clean = df_test['date']
clients_clean = df_test['client_id']
regions_clean = df_test['region']

print(X_test.head())

In [ ]:
try:
    if isinstance(model, dict) and 'model' in model:
        predictor = model['model']
        print(f"Usando modelo: {model.get('model_name', 'n/a')}")
    else:
        predictor = model

    y_pred = predictor.predict(X_test)
    
except Exception as e:
    print(f"erro: {e}")
    
    if isinstance(model, dict):
        for key in model.keys():
            try:
                predictor = model[key]
                if hasattr(predictor, 'predict'):
                    y_pred = predictor.predict(X_test)
                    break
            except Exception as e2:
                print(f"erro")
                continue
        else:
            raise Exception("erro")
    else:
        raise e

In [ ]:
mae = mean_absolute_error(y_true, y_pred)
rmse = np.sqrt(mean_squared_error(y_true, y_pred))
r2 = r2_score(y_true, y_pred)

print(f"Métricas do Modelo:")
print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R²: {r2:.3f}")

In [ ]:
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Previsões vs Real', 'Distribuição dos Erros', 'Consumo por Região', 'Evolução Temporal'),
    specs=[[{"type": "scatter"}, {"type": "histogram"}],
           [{"type": "box"}, {"type": "scatter"}]]
)

# Previsões vs Real
fig.add_trace(
    go.Scatter(x=y_true, y=y_pred, mode='markers', name='Previsões', 
               marker=dict(color='blue', opacity=0.6)),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=[y_true.min(), y_true.max()], y=[y_true.min(), y_true.max()], 
               mode='lines', name='Linha Ideal', line=dict(color='red', dash='dash')),
    row=1, col=1
)

# Distribuição dos Erros
errors = y_pred - y_true
fig.add_trace(
    go.Histogram(x=errors, name='Erros', nbinsx=30),
    row=1, col=2
)

# Consumo por Região
for region in regions_clean.unique():
    mask_region = regions_clean == region
    fig.add_trace(
        go.Box(y=y_true[mask_region], name=f'{region}'),
        row=2, col=1
    )

# Evolução Temporal
sample_client = clients_clean.iloc[0]
mask_client = clients_clean == sample_client
fig.add_trace(
    go.Scatter(x=dates_clean[mask_client], y=y_true[mask_client], 
               mode='lines', name='Real', line=dict(color='blue')),
    row=2, col=2
)
fig.add_trace(
    go.Scatter(x=dates_clean[mask_client], y=y_pred[mask_client], 
               mode='lines', name='Previsto', line=dict(color='red', dash='dash')),
    row=2, col=2
)

fig.update_layout(height=800, title_text="Dashboard - Análise de Previsões de Consumo Energético")
fig.show()

In [ ]:
region_stats = []
for region in regions_clean.unique():
    mask = regions_clean == region
    region_mae = mean_absolute_error(y_true[mask], y_pred[mask])
    region_r2 = r2_score(y_true[mask], y_pred[mask])
    region_stats.append({
        'Região': region,
        'MAE': region_mae,
        'R²': region_r2,
        'Consumo Médio Real': y_true[mask].mean(),
        'Consumo Médio Previsto': y_pred[mask].mean()
    })

region_df = pd.DataFrame(region_stats)
display(region_df.round(3))

In [ ]:
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('MAE por Região', 'R² por Região')
)

fig.add_trace(
    go.Bar(x=region_df['Região'], y=region_df['MAE'], name='MAE'),
    row=1, col=1
)

fig.add_trace(
    go.Bar(x=region_df['Região'], y=region_df['R²'], name='R²'),
    row=1, col=2
)

fig.update_layout(height=400, title_text="Performance do Modelo por Região")
fig.show()